# Subida, limpieza y carga a base de datos
En este Notebook se encuentra el código que carga los datos en bruto desde un Excel a un Dataframe Pandas. Después limpiamos los datos para finalmente descargarlos en formato .csv y además creamos un Script SQL DDL que al ejecutarlo carga los datos a PostgreSQL.

In [1]:
import pandas as pd
import numpy as np

Aqui tenemos la declaración de las funciones que vamos a usar

In [2]:
def separate_commas(dataframe, column):
    """Recoge en una lista todas las categorias de una columna de un dataframe separada por comas"""
    values = list()
    raw = dataframe[column].unique().tolist()
    for item in raw:
        if type(item) == str:
            separated = item.split(", ")
            for i in separated:
                if i not in values:
                    values.append(i)
    return values

def string_to_index(dataframe, col, reference, index):
    """Cambia los valores de una columna por los indices correspondientes de otra"""
    for value in dataframe[col].unique().tolist():
        i = reference.loc[reference[col] == value, index].values[0]
        dataframe.loc[dataframe[col] == value, col] = i
    return dataframe[col]

def explode_dataframe(relational, reference, column):
    """Takes a non-normalized relational dataframe and explodes it so it is in a N:N relational table style"""
    relational[column] = relational[column].apply(lambda x: x.split(', ') if type(x) == str else np.nan)
    relational = relational.explode(column).reset_index()
    j = 0
    for i, row in relational.iterrows():
        index = reference.loc[reference[column] == row[column], "index"]
        relational.at[i, 'index'] = j
        j += 1
        if not index.empty:
            relational.at[i, column] = index.values[0]
    return relational

## Carga del fichero

En este paso hacemos la carga del excel con los datos en bruto a un dataframe. Además, filtramos los datos para quedarnos únicamente con los que pertenecen a la comunidad de Madrid y solo con las columnas que vamos a usar

In [3]:
raw = pd.read_excel("raw_airbnb.xlsx", header=0) # Carga del excel
raw = raw[raw["State"].str.contains("madrid|Madrid",na=False)]
raw = raw[["ID", "Zipcode", "Latitude", "Longitude", "Accommodates", "Bathrooms", "Bedrooms", "Beds", "Price", "Security Deposit", "Cleaning Fee", "Guests Included", "Extra People", "Minimum Nights", "Maximum Nights", "Property Type", "Room Type", "Bed Type", "Cancellation Policy", "Host ID", "Host Since", "Host Response Time", "Host Response Rate", "Host Listings Count", "Number of Reviews", "First Review", "Last Review", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness", "Review Scores Checkin", "Review Scores Communication", "Review Scores Location", "Review Scores Value", "Reviews per Month", "Amenities", "Features", "Neighbourhood Group Cleansed", "Neighbourhood Cleansed"]]

C:\Users\sanmo\AppData\Local\Temp\ipykernel_12384\687493240.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  raw = pd.read_excel("raw_airbnb.xlsx", header=0) # Carga del excel


## Corrección del tipo de dato

Hacemos la exploración inicial de los tipos de dato, para poder modificarlos si fuese necesario. Podemos ver que muchos de los datos númericos aparecen como float cuando deberian ser integers. Además las fechas aparecen como texto en vez de fechas

In [4]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13208 entries, 0 to 13759
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   ID                            13208 non-null  int64         
 1   Zipcode                       12766 non-null  object        
 2   Latitude                      13208 non-null  float64       
 3   Longitude                     13208 non-null  float64       
 4   Accommodates                  13208 non-null  int64         
 5   Bathrooms                     13159 non-null  float64       
 6   Bedrooms                      13185 non-null  float64       
 7   Beds                          13159 non-null  float64       
 8   Price                         13199 non-null  float64       
 9   Security Deposit              5613 non-null   float64       
 10  Cleaning Fee                  7809 non-null   float64       
 11  Guests Included             

Modificamos los tipos de dato de las columnas que necesitamos.

In [5]:
raw["Zipcode"] = raw["Zipcode"].astype("float").astype('Int64')
raw["Bedrooms"] = raw["Bedrooms"].astype('Int64')
raw["Beds"] = raw["Beds"].astype('Int64')
raw["Price"] = raw["Price"].astype('Int64')
raw["Security Deposit"] = raw["Security Deposit"].astype('Int64')
raw["Cleaning Fee"] = raw["Cleaning Fee"].astype('Int64')
raw["Host Response Rate"] = raw["Host Response Rate"].astype('Int64')
raw["Host Listings Count"] = raw["Host Listings Count"].astype('Int64')
raw["Review Scores Rating"] = raw["Review Scores Rating"].astype('Int64')
raw["Review Scores Accuracy"] = raw["Review Scores Accuracy"].astype('Int64')
raw["Review Scores Cleanliness"] = raw["Review Scores Cleanliness"].astype('Int64')
raw["Review Scores Checkin"] = raw["Review Scores Checkin"].astype('Int64')
raw["Review Scores Communication"] = raw["Review Scores Communication"].astype('Int64')
raw["Review Scores Location"] = raw["Review Scores Location"].astype('Int64')
raw["Review Scores Value"] = raw["Review Scores Value"].astype('Int64')
raw["Host Since"] = pd.to_datetime(raw["Host Since"], format='%y%m%d')
raw["Host Since"] = pd.to_datetime(raw["Host Since"], format='%y%m%d')
raw["First Review"] = pd.to_datetime(raw["First Review"], format='%y%m%d')
raw["Last Review"] = pd.to_datetime(raw["Last Review"], format='%y%m%d')

## Limpieza de datos
Ahora vamos a limpiar algunos datos que están sucios, antes de dividirlos en las tablas necesarias.

Podemos ver que algunos Zipcodes están mal, ya que en España los Zipcodes tienen 5 cifras. Vamos a eliminar aquellos que no tengan el formato correcto

In [6]:
print(raw["Zipcode"].unique())
for i, row in raw.iterrows():
    if raw.at[i, 'Zipcode'] is not pd.NA and raw.at[i, 'Zipcode'] < 10000:
         raw.at[i, "Zipcode"] = np.nan

<IntegerArray>
[  <NA>,  28045,  28005,  28012,  28019,  28047,  28004,  28007,  28014,
  28042,  28055,  28025,  28044,  28054,  28026,  28013,  28033,  28008,
  28018,  28015,  28011, 280013,  27013,  28001,  25008,  20013,  28034,
  28016,  28056,   2015,  27004,   2805,  28094,   2804,  28003,  28010,
  28036,  28046,  28002,  28006,  28020,  28105,   2815,  28039,  28043,
  28027,  28017,  28850,     28,  28049,  28290,  28035,  28029,  28730,
  28050,  28024,  28660,  28023,  28040,  28224,  28030,  28053,  28038,
  28031,  28058,  28048,  28028,  28009,  28037,  28032,  28022,  28060,
  20126,  28041,  28052,  28830,  28051,  28021]
Length: 78, dtype: Int64


Por otra parte, vamos a ver la cantidad de nulos que existen en este dataset analizando el porcentaje de cada columna con nulos para después aplicar distintas tecnicas para limpiarlos.

In [7]:
total = raw.isnull().sum().sort_values(ascending=False)
porcentaje = (raw.isnull().sum()/raw.isnull().count()).sort_values(ascending=False)
nulos = pd.concat([total, porcentaje], axis=1, keys=['# de Nulos', 'Porcentaje'])
nulos.head(20)

,# de Nulos,Porcentaje
Security Deposit,7595,0.575030
Cleaning Fee,5399,0.408767
Review Scores Value,2871,0.217368
Review Scores Location,2871,0.217368
Review Scores Checkin,2869,0.217217
Review Scores Accuracy,2859,0.216460
Review Scores Cleanliness,2853,0.216005
Review Scores Communication,2853,0.216005
Review Scores Rating,2841,0.215097
Last Review,2717,0.205709


- Vemos que la mayoria de los nulos corresponden a Security Deposit y Cleaning Fee, que tienen casi la mitad de los datos como nulos, estos se  pueden poner a 0, como también otras variables numericas

- En cuanto a los zipcodes, vamos a intentar rellenarlos con los datos que tenemos, vamos a ordenar el dataset por Neighbourhood y vamos a rellenar los nulos con el dato inmediantemanete posterior de la columna.

- Vemos que hay 9 listings sin precio, vamos a eliminar esas filas

- Además, para un análisis posterior del precio, vamos a crear una columna nueva que sea el logaritmo del precio.

In [8]:
raw.fillna({'Security Deposit': 0, 'Cleaning Fee': 0, 'Bedrooms' : 0, 'Bathrooms': 0, 'Beds': 0, 'Reviews per Month': 0}, inplace=True)
raw['Zipcode'] = raw.sort_values('Neighbourhood Cleansed').fillna(method='bfill')['Zipcode']
raw = raw.dropna(subset=['Price'])
raw['LogPrice'] = np.log(raw['Price'])
raw.reset_index(inplace=True)

## Normalización de las tablas

Creamos sub tablas con los datos necesarios en cada una.

In [9]:
listing = raw[["ID", "Zipcode", "Latitude", "Longitude", "Accommodates", "Bathrooms", "Bedrooms", "Beds", "Price", "Security Deposit", "Cleaning Fee", "Guests Included", "Extra People", "Minimum Nights", "Maximum Nights", "Property Type", "Room Type", "Bed Type", "Cancellation Policy", "Neighbourhood Cleansed", "Host ID"]].copy()
host = raw[["Host ID", "Host Since", "Host Response Time", "Host Response Rate", "Host Listings Count"]].copy().drop_duplicates(keep='first')
host.reset_index(inplace=True)
reviews = raw[["Number of Reviews", "First Review", "Last Review", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness", "Review Scores Checkin", "Review Scores Communication", "Review Scores Location", "Review Scores Value", "Reviews per Month", "ID"]].copy().reset_index()

Muchas tablas van a ser básicamente "Look-up-tables" donde necesitamos guardar solamente un registro de cada valor

In [10]:
amenities = pd.DataFrame(separate_commas(raw, "Amenities"), columns = ['Amenities']).reset_index()
features = pd.DataFrame(separate_commas(raw, "Features"), columns = ['Features']).reset_index()
property_type = pd.DataFrame(raw["Property Type"].unique(), columns = ['Property Type']).reset_index()
room_type = pd.DataFrame(raw["Room Type"].unique(), columns = ['Room Type']).reset_index()
bed_type = pd.DataFrame(raw["Bed Type"].unique(), columns = ['Bed Type']).reset_index()
response_time = pd.DataFrame(raw["Host Response Time"].unique(), columns = ['Host Response Time']).reset_index()
cancellation_policy = pd.DataFrame(raw["Cancellation Policy"].unique(), columns = ['Cancellation Policy']).reset_index()
neighbourhood_group = pd.DataFrame(raw["Neighbourhood Group Cleansed"].unique(), columns = ['Neighbourhood Group Cleansed']).reset_index()

In [11]:
neighbourhood = raw[["Neighbourhood Cleansed", "Neighbourhood Group Cleansed"]].copy()
neighbourhood["Neighbourhood Group Cleansed"] = string_to_index(raw[["Neighbourhood Cleansed", "Neighbourhood Group Cleansed"]].copy(), "Neighbourhood Group Cleansed", neighbourhood_group, "index")
neighbourhood = neighbourhood.reset_index()
neighbourhood = neighbourhood.drop_duplicates(subset=["Neighbourhood Cleansed", "Neighbourhood Group Cleansed"], keep='first')
i = 0
for j, row in neighbourhood.iterrows():
    neighbourhood.at[j, 'index'] = i
    i += 1
neighbourhood = neighbourhood.reset_index(drop=True)

In [12]:
amenities_listing = raw[["ID", "Amenities"]].copy()
features_listing = raw[["ID", "Features"]].copy()

Para normalizar las tablas, reemplazamos aquellos datos que tenemos en otra tabla con sus ID correspondientes (Relaciones 1:N y 1:1).

In [13]:
listing["Property Type"] = string_to_index(listing, "Property Type", property_type, "index")
listing["Room Type"] = string_to_index(listing, "Room Type", room_type, "index")
listing["Bed Type"] = string_to_index(listing, "Bed Type", bed_type, "index")
listing["Cancellation Policy"] = string_to_index(listing, "Cancellation Policy", cancellation_policy, "index")
listing["Neighbourhood Cleansed"] = string_to_index(listing, "Neighbourhood Cleansed", neighbourhood, "index")
listing["Host ID"] = string_to_index(listing, "Host ID", host, "Host ID")

In [14]:
for i, row in host.iterrows():
    index = response_time.loc[response_time["Host Response Time"] == row["Host Response Time"], "index"]
    if not index.empty:
        host.at[i, "Host Response Time"] = index.values[0]

In [15]:
listing["review_id"] = np.nan
for i, row in listing.iterrows():
    j = listing.iloc[i]["ID"]
    review_id = reviews.loc[reviews["ID"] == j, 'index'].values[0]
    listing.at[i, "review_id"] = review_id
del reviews["ID"]
listing["review_id"] = listing["review_id"].astype('Int64')

Finalmente creamos dataframes para las tablas relacionales N:N

In [16]:
amenities_listing = explode_dataframe(amenities_listing, amenities, "Amenities")
features_listing = explode_dataframe(features_listing, features, "Features")

Descargamos los datos en CSV, donde cada csv es una de las tablas creadas, y también descargamos la tabla original ya limpia.

In [17]:
raw.to_csv("00_cleaned_airbnb.csv", index=False, encoding='utf-8')

In [18]:
listing.to_csv("01_listings.csv", index=False, encoding='utf-8')
host.to_csv("02_hosts.csv", index=False, encoding='utf-8')
reviews.to_csv("03_reviews.csv", index=False, encoding='utf-8')
amenities.to_csv("04_amenities.csv", index=False, encoding='utf-8')
features.to_csv("05_features.csv", index=False, encoding='utf-8')
property_type.to_csv("06_property_types.csv", index=False, encoding='utf-8')
room_type.to_csv("07_room_types.csv", index=False, encoding='utf-8')
bed_type.to_csv("08_bed_types.csv", index=False, encoding='utf-8')
response_time.to_csv("09_response_times.csv", index=False, encoding='utf-8')
cancellation_policy.to_csv("10_cancellation_policies.csv", index=False, encoding='utf-8')
neighbourhood_group.to_csv("11_neighbourhood_groups.csv", index=False, encoding='utf-8')
neighbourhood.to_csv("12_neighbourhoods.csv", index=False, encoding='utf-8')
amenities_listing.to_csv("13_relational_amenities_listing.csv", index=False, encoding='utf-8')
features_listing.to_csv("14_relational_features_listing.csv", index=False, encoding='utf-8')

In [19]:
open("data_upload.sql", "w").close()
def end_sql_sentence(i, df, script):
    if i == len(df.index) - 1:
        script.write(";\n\n")
    else:
        script.write(",\n")
    
with open("data_upload.sql", "a",encoding='utf-8') as script:
    script.write("INSERT INTO\n\tgrupo_15.neighbourhood_groups(id, neighbourhood_group)\nVALUES\n")
    for i, row in neighbourhood_group.iterrows():
        script.write(f"\t({row['index']}, '{row['Neighbourhood Group Cleansed']}')")
        end_sql_sentence(i, neighbourhood_group, script)

    script.write("INSERT INTO\n\tgrupo_15.neighbourhoods(id, neighbourhood, id_neighbourhood_group)\nVALUES\n")
    for i, row in neighbourhood.iterrows():
        script.write(f"\t({row['index']}, '{row['Neighbourhood Cleansed']}', {row['Neighbourhood Group Cleansed']})")
        end_sql_sentence(i, neighbourhood, script)

    script.write("INSERT INTO\n\tgrupo_15.property_types(id, property_type)\nVALUES\n")
    for i, row in property_type.iterrows():
        script.write(f"\t({row['index']}, '{row['Property Type']}')")
        end_sql_sentence(i, property_type, script)
    
    script.write("INSERT INTO\n\tgrupo_15.room_types(id, room_type)\nVALUES\n")
    for i, row in room_type.iterrows():
        script.write(f"\t({row['index']}, '{row['Room Type']}')")
        end_sql_sentence(i, room_type, script)
    
    script.write("INSERT INTO\n\tgrupo_15.bed_types(id, bed_type)\nVALUES\n")
    for i, row in bed_type.iterrows():
        script.write(f"\t({row['index']}, '{row['Bed Type']}')")
        end_sql_sentence(i, bed_type, script)
    
    script.write("INSERT INTO\n\tgrupo_15.response_times(id, response_time)\nVALUES\n")
    for i, row in response_time.iterrows():
        script.write(f"\t({row['index']}, '{row['Host Response Time']}')")
        end_sql_sentence(i, response_time, script)
    
    script.write("INSERT INTO\n\tgrupo_15.hosts(id, since, id_response_time, response_rate, listings_count)\nVALUES\n")
    for i, row in host.iterrows():
        script.write(f"\t({row['Host ID']}, '{row['Host Since']}', {row['Host Response Time']}, {row['Host Response Rate']}, {row['Host Listings Count']})")
        end_sql_sentence(i, host, script)

    script.write("INSERT INTO\n\tgrupo_15.cancellation_policies(id, cancellation_policy)\nVALUES\n")
    for i, row in cancellation_policy.iterrows():
        script.write(f"\t({row['index']}, '{row['Cancellation Policy']}')")
        end_sql_sentence(i, cancellation_policy, script)

    script.write("INSERT INTO\n\tgrupo_15.features(id, feature)\nVALUES\n")
    for i, row in features.iterrows():
        script.write(f"\t({row['index']}, '{row['Features']}')")
        end_sql_sentence(i, features, script)
    
    script.write("INSERT INTO\n\tgrupo_15.amenities(id, amenity)\nVALUES\n")
    for i, row in amenities.iterrows():
        script.write(f"\t({row['index']}, '{row['Amenities']}')")
        end_sql_sentence(i, amenities, script)

    script.write("INSERT INTO\n\tgrupo_15.reviews(id, number_reviews, first_review, last_review, scores_rating, scores_accuracy, scores_cleanliness, scores_checkin, scores_communication, scores_location, scores_value, reviews_per_month)\nVALUES\n")
    for i, row in reviews.iterrows():
        script.write(f"\t({row['index']}, {row['Number of Reviews']}, '{row['First Review']}', '{row['Last Review']}', {row['Review Scores Rating']}, {row['Review Scores Accuracy']}, {row['Review Scores Cleanliness']}, {row['Review Scores Checkin']}, {row['Review Scores Communication']}, {row['Review Scores Location']}, {row['Review Scores Value']}, {row['Reviews per Month']})")
        end_sql_sentence(i, reviews, script)

    script.write("INSERT INTO\n\tgrupo_15.listings(id, id_host, id_neighbourhood, zipcode, latitude, longitude, id_property_type, id_room_type, id_bed_type, accommodates, bathrooms, bedrooms, beds, price, security_deposit, cleaning_fee, extra_people, minimum_nights, maximum_nights, id_cancellation_policy, id_review)\nVALUES\n")
    for i, row in listing.iterrows():
        script.write(f"\t({row['ID']},  {row['Host ID']}, {row['Neighbourhood Cleansed']}, {row['Zipcode']}, {row['Latitude']}, {row['Longitude']}, {row['Property Type']}, {row['Room Type']}, {row['Bed Type']}, {row['Accommodates']}, {row['Bathrooms']}, {row['Bedrooms']}, {row['Beds']}, {row['Price']}, {row['Security Deposit']}, {row['Cleaning Fee']}, {row['Extra People']}, {row['Minimum Nights']}, {row['Maximum Nights']}, {row['Cancellation Policy']}, {row['review_id']})")
        end_sql_sentence(i, listing, script)

    script.write("INSERT INTO\n\tgrupo_15.rel_amenities_listings(id, id_amenity, id_listing)\nVALUES\n")
    for i, row in amenities_listing.iterrows():
        script.write(f"\t({row['index']}, {row['Amenities']}, {row['ID']})")
        end_sql_sentence(i, amenities_listing, script)
        
    script.write("INSERT INTO\n\tgrupo_15.rel_features_listings(id, id_feature, id_listing)\nVALUES\n")
    for i, row in features_listing.iterrows():
        script.write(f"\t({row['index']}, {row['Features']}, {row['ID']})")
        end_sql_sentence(i, features_listing, script)
with open("data_upload.sql", "r") as file:
    filedata = file.read()
    filedata = filedata.replace("<NA>", "NULL").replace("nan", "NULL").replace("NaT", "NULL").replace("'NULL'", "NULL").replace("00:00:00", "")
with open("data_upload.sql", 'w') as file:
    file.write(filedata)